In [1]:
! pip install pyspark
! pip install setuptools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 33.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840628 sha256=99eca8dd581b1b1df7fba221babde7cee45c3b08f95edd4280ebf0c684f81065
  Stored in directory: /Users/maksimlyara/Library/Caches/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession


In [3]:
df_logs_5 = pd.read_parquet('logs_df_2024-08-05.parquet', engine='pyarrow')
df_logs_5.head(3)

,event_timestamp,user_id,region,city,video_id,watchtime
0,2024-08-05 12:12:32+03:00,bd1b6b07-e6a2-46ad-aa58-990bb7e89252,288eb079-1c29-4749-ab7e-550451a4d765,da903ff0-9eb8-4e91-b25d-22c70f962ca2,0af133e1-4fbf-4e10-b9e8-19fbe1a011c3,32
1,2024-08-05 18:30:15+03:00,7db026fc-b55c-4580-bc26-e3df59ba6c7c,f58ab76f-2581-447a-b95b-5832e8757054,8449a572-fd10-44ae-a899-af656e350ad9,4f42aab2-6238-4381-bb53-36213916e912,340
2,2024-08-05 12:35:16+03:00,5c94a654-f376-4529-b463-b62b6b50e8ca,2c1f4ce4-66e0-4481-a0ed-244bf76fa83a,93a12888-fdf4-4889-a1ef-3ef0537d674d,8b106b84-f3f5-4c27-820c-7e1546b43614,12


In [4]:
df_video = pd.read_parquet('video_stat.parquet', engine= 'pyarrow')
df_video.head(3)

,video_id,v_pub_datetime,v_total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,...,v_category_popularity_percent_7_days,v_category_popularity_percent_30_days,v_long_views_1_days,v_long_views_7_days,v_long_views_30_days,row_number,title,description,category_id,author_id
0,973f24c0-15b2-434f-8740-4e2726f79c30,2024-06-15 22:58:03+03:00,0,556,2,0,0,0,0,7.733,...,1.656386,1.616750,0,0,0,1,пережила свой яд,,Мультфильмы,79276150-fdaf-4626-9452-6f1ad975741d
1,1a3f527c-44b2-4f7e-9641-26f078edbf2e,2021-04-18 17:27:42+03:00,0,109,2,1,0,0,0,379.000,...,5.570802,5.649635,0,0,0,1,"🎓Знаете ли Вы, что ؟ Интересные факты ! # ...","🎓Знаете ли Вы, что ? Интересные факты ! # 2🎓...",Развлечения,16278647-5912-48af-a071-582dcfd0e354
2,2f563d09-45ef-4e27-8da0-daa74235a2e7,2022-11-17 16:36:43+03:00,0,163,2,2,0,0,0,1020.767,...,0.342614,0.285310,0,0,0,1,"ЗИМНЯЯ, ТЕПЛАЯ ШЛЯПА крючком. Как связать поля...",Шляпа связана из пряжи NAKO Sport Wool в 100гр...,Хобби,9afa94aa-c2a6-43fc-a0d1-175eaf7fa194


In [5]:
df_video['category_id'].unique()


array(['Мультфильмы', 'Развлечения', 'Хобби', 'Животные', 'Разное',
       'Лайфстайл', 'Телепередачи', 'Спорт', 'Недвижимость', 'Сериалы',
       'Детям', 'Музыка', 'Юмор', 'Авто-мото', 'Сад и огород', 'Фильмы',
       'Видеоигры', 'Наука', 'Путешествия', 'Технологии и интернет',
       'Лайфхаки', 'Культура', 'Обзоры и распаковки товаров', 'Обучение',
       'Здоровье', 'Охота и рыбалка', 'Строительство и ремонт', 'Аниме',
       'Эзотерика', 'Еда', 'UNKNOWN', 'Интервью',
       'Бизнес и предпринимательство', 'Техника и оборудование',
       'Психология', 'Красота', 'Природа', 'Дизайн', 'Аудиокниги',
       'Аудио'], dtype=object)

In [6]:

# Создаем SparkSession
spark = SparkSession.builder \
    .appName('VideoDataProcessing') \
    .getOrCreate()

# Загружаем parquet-файлы в PySpark DataFrame
df_video = spark.read.parquet('video_stat.parquet')
df_logs_5 = spark.read.parquet('logs_df_2024-08-05.parquet')

# Объединение данных по 'video_id'
df_video_add_watchtime = df_video.join(df_logs_5.select('video_id', 'watchtime'), on='video_id', how='left')

# (Необязательно) Если нужно вывести результат в pandas DataFrame для дальнейшей обработки:
# Пример простой агрегации и анализа данных в PySpark
df_video_add_watchtime.groupBy('category_id').agg({'watchtime': 'avg'}).show()


+--------------------+------------------+
|         category_id|    avg(watchtime)|
+--------------------+------------------+
|               Детям|401.50759409595025|
|            Культура|2629.7981086676737|
|           Лайфстайл| 165.7867915394168|
|         Мультфильмы| 514.7748540913898|
|              Музыка|  415.812855224911|
|               Аудио| 661.9897807965422|
|            Животные| 73.40577332675622|
|                Юмор| 300.3663552818508|
|              Фильмы| 1927.277600445502|
|        Сад и огород|134.32996525164535|
|               Хобби|121.81678718911748|
|         Путешествия| 223.7675925243291|
|            Лайфхаки| 209.3924483629181|
|           Авто-мото| 247.4899499639104|
|               Аниме| 575.6136564025228|
|Обзоры и распаков...| 72.07416529180662|
|                 Еда| 71.22446479986871|
|Бизнес и предприн...|131.55275324459924|
|               Наука|257.57630928684193|
|           Эзотерика|207.22435100338606|
+--------------------+------------

In [7]:
df_video_add_watchtime

DataFrame[video_id: string, v_pub_datetime: timestamp, v_total_comments: decimal(20,0), v_year_views: decimal(20,0), v_month_views: decimal(20,0), v_week_views: decimal(20,0), v_day_views: decimal(20,0), v_likes: decimal(20,0), v_dislikes: decimal(20,0), v_duration: double, v_cr_click_like_7_days: double, v_cr_click_dislike_7_days: double, v_cr_click_vtop_7_days: double, v_cr_click_long_view_7_days: double, v_cr_click_comment_7_days: double, v_cr_click_like_30_days: double, v_cr_click_dislike_30_days: double, v_cr_click_vtop_30_days: double, v_cr_click_long_view_30_days: double, v_cr_click_comment_30_days: double, v_cr_click_like_1_days: double, v_cr_click_dislike_1_days: double, v_cr_click_vtop_1_days: double, v_cr_click_long_view_1_days: double, v_cr_click_comment_1_days: double, v_is_hidden: smallint, v_is_deleted: smallint, v_avg_watchtime_1_day: double, v_avg_watchtime_7_day: double, v_avg_watchtime_30_day: double, v_frac_avg_watchtime_1_day_duration: double, v_frac_avg_watchtime_